In [8]:
import json

from collections import Counter

In [9]:
with open('../public/static/bike_routes.geojson') as fh:
    data = json.loads(fh.read())

In [19]:
categories = Counter()
lengths = Counter()

for feature in data['features']:
    categories[feature['properties']['bike_network_category']] += 1
    if feature['geometry']:
        lengths[len(feature.get('geometry', {}).get('coordinates'))] += 1

In [12]:
categories

Counter({'Proposed Bike+ - Non-Arterial': 3076,
         'Existing Bike+ - Non-Arterial': 804,
         'Non-Bike+ Planned': 250,
         'Proposed Multi-Use Trail': 175,
         'Existing Bike+ - Arterial': 525,
         'Proposed Bike+ - Arterial': 1503,
         'Proposed Bike+ - Upgrade, Arterial': 1464,
         'Existing Multi-Use Trail': 337,
         '': 136,
         'Non-Bike+': 197})

In [21]:
!pip install pyhigh

     |████████████████████████████████| 19.8MB 16.2MB/s eta 0:00:01
  Running setup.py install for pyhigh ... done
You should consider upgrading via the 'pip install --upgrade pip' command.


In [22]:
data['features'][0]

{'type': 'Feature',
 'id': 1,
 'geometry': {'type': 'LineString',
  'coordinates': [[-122.308954277912, 47.604327437898],
   [-122.307649047488, 47.6043216339769]]},
 'properties': {'OBJECTID': 1,
  'id': 23677,
  'artclass_desc': 'Not Designated',
  'stname_ord': 'E ALDER ST',
  'bike_network_category': 'Proposed Bike+ - Non-Arterial'}}

In [23]:
from pyhigh import get_elevation

/Users/abrahamepton/venvs/maps/lib/python3.8/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [29]:
get_elevation(lat=47.604327437898, lon=-122.308954277912) * 3.28084

314.96064

In [28]:
get_elevation(lat=47.6043216339769, lon=-122.307649047488) * 3.28084

321.52232

In [27]:
get_elevation(lat=47.5541164, lon=-122.2896943) * 3.28084

144.35696